In [9]:
from unicodedata import numeric
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

In [10]:
# df = pd.read_csv('../data/datascience-cis-2022.csv')
# devices = ['bd:0d:8e:c5', 'bd:24:bb:1e', 'bd:0d:af:0b', 'ee:dd:34:1d', 'ee:dd:39:08', 'ee:dd:39:09']
# small_df = df.loc[df['device_mac'].isin(devices)]
# small_df.to_csv('../data/small_data.csv',index=False)

In [11]:
df = pd.read_csv('../data/small_data.csv')
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')

In [12]:
for col in df.columns:
    if (df[col] == 0).all():
        print(col)

FromInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount
FromInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount
ToInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount
ToInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount


In [14]:
def nonzero_proportion(x):
    return (x!=0).mean()

In [15]:
numeric_cols = list(df.columns)[:-1]
funcs = {'mean':np.mean, 'std':np.std, 'max':np.max, 'min':np.min, 'nonzero_p':nonzero_proportion}

In [16]:
df['time_group'] = df.index.to_period('H').strftime('%d_%H')

In [17]:
summary = {key: [] for key in funcs.keys()}
summary['info'] = []
summary['target'] = []

In [18]:
for time_slice in tqdm(df['time_group'].unique()):
    for device in (df['device_mac'].unique()):
        tmp = df.loc[(df['time_group']==time_slice)&(df['device_mac']==device)][numeric_cols]
        for col in tmp.columns:
            for k in funcs.keys():
                summary[k].append(funcs[k](tmp[col]))
            summary['info'].append(device+'_'+time_slice)
            summary['target'].append(col)

100%|██████████| 337/337 [00:48<00:00,  6.99it/s]


In [19]:
summary = pd.DataFrame(summary)
summary.to_csv('../data/transformed_data.csv',index=False)

In [26]:
target_mapper = {v:i for i,v in enumerate(sorted(summary['target'].unique()))}

In [27]:
import json

In [28]:
with open('target_mappings.json','w') as f:
    json.dump(target_mapper, f)